Skrypt do automatycznego pobierania danych meteorologicznych ze strony https://www.wunderground.com/.
Niestety rok trzeba zmieniać manualnie po każdej iteracji.
Przed uruchomieniem skryptu, należy utworzyć pusty plik csv o nazwie "{wybrany rok}_historic_data.csv".

In [ ]:
from selenium import webdriver
from time import sleep

#niestety, aby atomatycznie ściągnąć dane należy pobrać chromedriver ze strony https://sites.google.com/chromium.org/driver/
#i umieścić plik pod adresem 'C:\Program Files\chromedriver.exe'

browser = webdriver.Chrome('C:\Program Files\chromedriver.exe')
browser.get('https://www.wunderground.com/history/monthly/EPGD/date/2015-1')

avr_temp_F = '/html/body/app-root/app-history/one-column-layout/wu-header/sidenav/mat-sidenav-container/mat-sidenav-content/div/section/div[2]/div[1]/div[3]/div[1]/div/lib-city-history-summary/div/div[2]/table/tbody[1]/tr[2]/td[2]'
max_temp_F = '/html/body/app-root/app-history/one-column-layout/wu-header/sidenav/mat-sidenav-container/mat-sidenav-content/div/section/div[2]/div[1]/div[3]/div[1]/div/lib-city-history-summary/div/div[2]/table/tbody[1]/tr[1]/td[1]'
min_temp_F = '/html/body/app-root/app-history/one-column-layout/wu-header/sidenav/mat-sidenav-container/mat-sidenav-content/div/section/div[2]/div[1]/div[3]/div[1]/div/lib-city-history-summary/div/div[2]/table/tbody[1]/tr[3]/td[3]'
avr_wind_mph = '/html/body/app-root/app-history/one-column-layout/wu-header/sidenav/mat-sidenav-container/mat-sidenav-content/div/section/div[2]/div[1]/div[3]/div[1]/div/lib-city-history-summary/div/div[2]/table/tbody[4]/tr[1]/td[2]'
avr_press_inch = '/html/body/app-root/app-history/one-column-layout/wu-header/sidenav/mat-sidenav-container/mat-sidenav-content/div/section/div[2]/div[1]/div[3]/div[1]/div/lib-city-history-summary/div/div[2]/table/tbody[5]/tr/td[2]'

#print(browser.find_element_by_xpath(avr_temp_F).text)
#print(browser.find_element_by_xpath(max_temp_F).text)
#print(browser.find_element_by_xpath(min_temp_F).text)
#print(browser.find_element_by_xpath(avr_wind_mph).text)
#print(browser.find_element_by_xpath(avr_press_inch).text)

monthtonum = {
                "January" : "01",
                "February" : "02",
                "March" : "03",
                "April" : "04",
                "May" : "05",
                "June" : "06",
                "July" : "07",
                "August" : "08",
                "September" : "09",
                "October" : "10",
                "November" : "11",
                "December" : "12"
}


yearselector = '/html/body/app-root/app-history/one-column-layout/wu-header/sidenav/mat-sidenav-container/mat-sidenav-content/div/section/div[2]/div[1]/div[1]/div[1]/div/lib-date-selector/div/select[2]'
yearxpath = browser.find_element_by_xpath(yearselector)

#Zamiast 2015, można wpisać wybrany rok.
reqyear = '2015'

for year in yearxpath.find_elements_by_tag_name('option'):
    if year.text == reqyear:
        #print(f"found {reqyear}")
        year.click()

        
monthselector = '/html/body/app-root/app-history/one-column-layout/wu-header/sidenav/mat-sidenav-container/mat-sidenav-content/div/section/div[2]/div[1]/div[1]/div[1]/div/lib-date-selector/div/select[1]'
monthxpath = browser.find_element_by_xpath(monthselector)


reqmonth = 'January'

for month in monthxpath.find_elements_by_tag_name('option'):
    if month.text == reqmonth:
        #print(f"found {reqmonth}")
        month.click()
        break
browser.find_element_by_xpath('/html/body/app-root/app-history/one-column-layout/wu-header/sidenav/mat-sidenav-container/mat-sidenav-content/div/section/div[2]/div[1]/div[1]/div[1]/div/lib-date-selector/div/input').click()

sleep(2)

with open(f"{reqyear}_historic_data.csv", "w") as writer:
    writer.write('Year, avr_temp_F, max_temp_F, min_temp_F, avr_wind_mph, avr_press_inch\n')
    
    monthxpath = browser.find_element_by_xpath('/html/body/app-root/app-history/one-column-layout/wu-header/sidenav/mat-sidenav-container/mat-sidenav-content/div/section/div[2]/div[1]/div[1]/div[1]/div/lib-date-selector/div/select[1]')
    monthoption = monthxpath.find_elements_by_tag_name('option')
        
    currentmonthnum = 0
    for month in monthoption:
        print("Trying to refresh day object...")
        month = browser.find_element_by_xpath("/html/body/app-root/app-history/one-column-layout/wu-header/sidenav/mat-sidenav-container/mat-sidenav-content/div/section/div[2]/div[1]/div[1]/div[1]/div/lib-date-selector/div/select[1]").find_elements_by_tag_name("option")[currentmonthnum]
        view = browser.find_element_by_xpath("/html/body/app-root/app-history/one-column-layout/wu-header/sidenav/mat-sidenav-container/mat-sidenav-content/div/section/div[2]/div[1]/div[1]/div[1]/div/lib-date-selector/div/input")
        currentmonthnum = int(f"{monthtonum[month.text]}")

        print("Trying to click on day and view")
        month.click()
        sleep(2.5)
        view.click()
        sleep(2.5)
        month = browser.find_element_by_xpath("/html/body/app-root/app-history/one-column-layout/wu-header/sidenav/mat-sidenav-container/mat-sidenav-content/div/section/div[2]/div[1]/div[1]/div[1]/div/lib-date-selector/div/select[1]").find_elements_by_tag_name("option")[currentmonthnum - 1]
        
        avr_temp_F = browser.find_element_by_xpath("/html/body/app-root/app-history/one-column-layout/wu-header/sidenav/mat-sidenav-container/mat-sidenav-content/div/section/div[2]/div[1]/div[3]/div[1]/div/lib-city-history-summary/div/div[2]/table/tbody[1]/tr[2]/td[2]").text

        max_temp_F = browser.find_element_by_xpath("/html/body/app-root/app-history/one-column-layout/wu-header/sidenav/mat-sidenav-container/mat-sidenav-content/div/section/div[2]/div[1]/div[3]/div[1]/div/lib-city-history-summary/div/div[2]/table/tbody[1]/tr[1]/td[1]").text

        min_temp_F = browser.find_element_by_xpath("/html/body/app-root/app-history/one-column-layout/wu-header/sidenav/mat-sidenav-container/mat-sidenav-content/div/section/div[2]/div[1]/div[3]/div[1]/div/lib-city-history-summary/div/div[2]/table/tbody[1]/tr[3]/td[3]").text

        avr_wind_mph = browser.find_element_by_xpath("/html/body/app-root/app-history/one-column-layout/wu-header/sidenav/mat-sidenav-container/mat-sidenav-content/div/section/div[2]/div[1]/div[3]/div[1]/div/lib-city-history-summary/div/div[2]/table/tbody[4]/tr[1]/td[2]").text

        avr_press_inch = browser.find_element_by_xpath("/html/body/app-root/app-history/one-column-layout/wu-header/sidenav/mat-sidenav-container/mat-sidenav-content/div/section/div[2]/div[1]/div[3]/div[1]/div/lib-city-history-summary/div/div[2]/table/tbody[5]/tr/td[2]").text

        writer.write(f"{reqyear},{avr_temp_F},{max_temp_F},{min_temp_F},{avr_wind_mph},{avr_press_inch}\n")
        